In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, json
from astropy import constants as const
from astropy import units as u

# increase label text size to 14
font = {'size': 14}
plt.rc('font', **font)

### Load data

In [48]:
data = pd.read_csv("materials/neutron_att_coeff.csv", sep=',')
print(data)

materials = data['material'].values

       material  molar_mass   density  scattering  absorption_2200
0      hydrogen     1.00794   0.08988      82.020          0.33260
1        oxygen    15.99940   1.42900       4.232          0.00019
2        copper    63.54600   8.96000       8.030          3.78000
3          lead   207.20000  11.35000      11.118          0.17100
4         water    18.01530   0.99700     168.000       1365.00000
5  polyethylene    14.02660   0.96000      -1.000         -1.00000


### Input parameters

In [49]:
# Energy of incoming radiation
neutron_energy = 25.30e-3 * u.eV    # 25.30 meV (2200m/s)
velocity = np.sqrt(2 * neutron_energy / const.m_n).to(u.m/u.s)
gamma_energy = 2.615e6 * u.eV       # Tl-208 peak @ 2.615 MeV

# Max count rate allowed
max_gamma_count_rate = 0.1 / u.yr   # 0.1 counts per year
max_neutron_count_rate = 0.1 / u.yr # 0.1 counts per year

# Background radiation levels
incoming_neutron_flux = 1.51e-7 / u.cm**2 / u.s 
incoming_gamma_counts = 0.013 / u.s

# Surface area of detector
surface_area = 5184 * u.cm**2

print("===== Input parameters =====")
print(f"Max neutron count rate: {max_neutron_count_rate.to(1/u.s)} = {max_neutron_count_rate}")
print(f"Incoming neutron flux: {incoming_neutron_flux}")
print(f"Neutron energy: {neutron_energy.to(u.MeV)} = {neutron_energy.to(u.eV)}")
print(f"Neutron velocity: {velocity}")
print()
print(f"Max gamma count rate: {max_gamma_count_rate.to(1/u.s)} = {max_gamma_count_rate}")
print(f"Incoming gamma counts: {incoming_gamma_counts}")
print(f"Gamma energy: {gamma_energy.to(u.MeV)} = {gamma_energy.to(u.eV)}")
print()
print(f"Detector surface area: {surface_area}")
print()

===== Input parameters =====
Max neutron count rate: 3.168808781402895e-09 1 / s = 0.1 1 / yr
Incoming neutron flux: 1.51e-07 1 / (cm2 s)
Neutron energy: 2.5299999999999998e-08 MeV = 0.0253 eV
Neutron velocity: 2200.049482932255 m / s

Max gamma count rate: 3.168808781402895e-09 1 / s = 0.1 1 / yr
Incoming gamma counts: 0.013 1 / s
Gamma energy: 2.6149999999999998 MeV = 2615000.0 eV

Detector surface area: 5184.0 cm2



In [50]:
# Calculate reduction required
incoming_counts = (incoming_neutron_flux * surface_area).to(1/u.yr)
reduction_required = (max_neutron_count_rate / incoming_counts).to(u.dimensionless_unscaled)
print(f"Incoming neutron counts: {incoming_counts.to(1/u.s)} = {incoming_counts}")
print(f"Reduction required: {reduction_required}")
print()

for material in materials:
    # Get material data
    material_data = data[data['material'] == material]

    density     = material_data['density'].values[0] * u.g / u.cm**3
    molar_mass  = material_data['molar_mass'].values[0] * u.g / u.mol
    scat_coeff  = material_data['scattering'].values[0] * u.barn
    abs_coeff   = material_data['absorption_2200'].values[0] * u.barn

    if (scat_coeff == -1 * u.barn or abs_coeff == -1 * u.barn):
        continue # Skip materials with no scattering or absorption data

    # Calculate number density
    number_density = (density * const.N_A / molar_mass).to(1/u.cm**3)

    # Scale neutron absorption coefficient
    scale_factor = ((2200 * u.m / u.s) / velocity).to(u.dimensionless_unscaled) # scales with 1/v
    abs_coeff = scale_factor * abs_coeff

    # Calculate total neutron attenuation coefficient
    total_coeff = (scat_coeff + abs_coeff).to(u.cm**2)

    # Calculate thickness required
    thickness_scat = (-(np.log(reduction_required))/(scat_coeff * number_density)).to(u.cm)
    thickness_abs = (-(np.log(reduction_required))/(abs_coeff * number_density)).to(u.cm)
    thickness = (-(np.log(reduction_required))/(total_coeff * number_density)).to(u.cm)

    print(f"===== Material: {material} =====")
    print(f"Number density: {number_density}")
    print(f"Absorption coefficient scale factor: {scale_factor}")
    print(f"Scaled absorption coefficient: {scat_coeff}")
    print(f"Scattering coefficient: {abs_coeff}")
    print(f"Total attenuation coefficient: {total_coeff}")
    print()
    print(f"Thickness required (scattering only): {thickness_scat}")
    print(f"Thickness required (absorption only): {thickness_abs}")
    print(f"Thickness required: {thickness}")
    print()

Incoming neutron counts: 0.0007827839999999999 1 / s = 24702.7843584 1 / yr
Reduction required: 4.048126662531293e-06

===== Material: hydrogen =====
Number density: 5.370061824203821e+22 1 / cm3
Absorption coefficient scale factor: 0.9999775082639554
Scaled absorption coefficient: 82.02 barn
Scattering coefficient: 0.33259251924859157 barn
Total attenuation coefficient: 8.235259251924859e-23 cm2

Thickness required (scattering only): 2.8192046041123104 cm
Thickness required (absorption only): 695.2386125571911 cm
Thickness required: 2.8078188500895713 cm

===== Material: oxygen =====
Number density: 5.37872616850632e+22 1 / cm3
Absorption coefficient scale factor: 0.9999775082639554
Scaled absorption coefficient: 4.232 barn
Scattering coefficient: 0.00018999572657015152 barn
Total attenuation coefficient: 4.23218999572657e-24 cm2

Thickness required (scattering only): 54.550728240287 cm
Thickness required (absorption only): 1215073.0233801096 cm
Thickness required: 54.54827929417223 c

In [51]:
# Calculate reduction required
reduction_required = (max_gamma_count_rate / incoming_gamma_counts).to(u.dimensionless_unscaled)
print(f"Incoming gamma counts: {incoming_gamma_counts.to(1/u.yr)} = {incoming_gamma_counts}")
print(f"Reduction required: {reduction_required}")
print()

for material in materials:
    # Get material data
    material_data = data[data['material'] == material]
    density = material_data['density'].values[0] * u.g / u.cm**3
    df = pd.read_csv(f"materials/gamma_att_coeff/{material}.csv", sep=',')
    energy = df['energy'].values * u.MeV
    coeff = df['coeff'].values * u.cm**2 / u.g

    # Calculate the gamma attenuation coefficient for 2.615 MeV
    gamma_att_coeff = np.interp(gamma_energy, energy, coeff)
    thickness = -(np.log(reduction_required))/(gamma_att_coeff * density)
    
    print(f"===== Material: {material} =====")
    print(f"Gamma attenuation coefficient: {gamma_att_coeff}")
    print(f"Density: {density}")
    print(f"Thickness required: {thickness}")
    print()


Incoming gamma counts: 410248.80000000005 1 / yr = 0.013 1 / s
Reduction required: 2.4375452164637654e-07

===== Material: hydrogen =====
Gamma attenuation coefficient: 0.0763248 cm2 / g
Density: 0.08988 g / cm3
Thickness required: 2219.6708582582573 cm

===== Material: oxygen =====
Gamma attenuation coefficient: 0.0392887 cm2 / g
Density: 1.429 g / cm3
Thickness required: 271.2173268896703 cm

===== Material: copper =====
Gamma attenuation coefficient: 0.0383231 cm2 / g
Density: 8.96 g / cm3
Thickness required: 44.34541024007643 cm

===== Material: lead =====
Gamma attenuation coefficient: 0.0437722 cm2 / g
Density: 11.35 g / cm3
Thickness required: 30.649478017209123 cm

===== Material: water =====
Gamma attenuation coefficient: 0.043436050000000004 cm2 / g
Density: 0.997 g / cm3
Thickness required: 351.6185966664464 cm

===== Material: polyethylene =====
Gamma attenuation coefficient: 0.04437315 cm2 / g
Density: 0.96 g / cm3
Thickness required: 357.45866252302915 cm



In [52]:
H_scat_coeff  = 82.02 * u.barn
H_abs_coeff   = 0.3326 * u.barn

# Calculate number density
number_density = 6.665528e22*(1/u.cm**3)

scat_coeff = H_scat_coeff

# Scale neutron absorption coefficient
scale_factor = ((2200 * u.m / u.s) / velocity).to(u.dimensionless_unscaled) # scales with 1/v
abs_coeff = scale_factor * H_abs_coeff

# Calculate total neutron attenuation coefficient
total_coeff = (scat_coeff + abs_coeff).to(u.cm**2)

# Calculate thickness required
thickness_scat = (-(np.log(reduction_required))/(scat_coeff * number_density)).to(u.cm)
thickness_abs = (-(np.log(reduction_required))/(abs_coeff * number_density)).to(u.cm)
thickness = (-(np.log(reduction_required))/(total_coeff * number_density)).to(u.cm)

print(f"===== Material: water =====")
print(f"Number density: {number_density}")
print(f"Absorption coefficient scale factor: {scale_factor}")
print(f"Scaled absorption coefficient: {scat_coeff}")
print(f"Scattering coefficient: {abs_coeff}")
print(f"Total attenuation coefficient: {total_coeff}")
print()
print(f"Thickness required (scattering only): {thickness_scat}")
print(f"Thickness required (absorption only): {thickness_abs}")
print(f"Thickness required: {thickness}")
print()

===== Material: water =====
Number density: 6.665528e+22 1 / cm3
Absorption coefficient scale factor: 0.9999775082639554
Scaled absorption coefficient: 82.02 barn
Scattering coefficient: 0.33259251924859157 barn
Total attenuation coefficient: 8.235259251924859e-23 cm2

Thickness required (scattering only): 2.785242394034479 cm
Thickness required (absorption only): 686.8632574022495 cm
Thickness required: 2.773993801170406 cm

